In [ ]:
import pickle

## 流利度

[参考](https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk)

### 训练

In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [ ]:
from nltk import word_tokenize, sent_tokenize 

In [ ]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = toktok.tokenize

In [ ]:
test_sentence = "This is a foobar sentence. Yes it is."
word_tokenize(test_sentence)

In [ ]:
import pandas as pd

In [ ]:
df_all_text = pd.read_csv('data/公开数据/Good Luck Charlie (2010)/all_text.csv')
df_train = pd.read_csv('data/after_preprocess/task1_charlie_train.csv')
sentences = df_train['Sentence'].tolist()+df_all_text['text'].tolist()
sentences = [x.lower() for x in sentences]
tokenized_text = [list(map(str.lower, word_tokenize(sent)))
                  for sent in sentences]

In [ ]:
len(tokenized_text)

In [ ]:
sentences[0],tokenized_text[0]

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline

In [ ]:
n = 2

In [ ]:
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [ ]:
from nltk.lm import MLE
model = MLE(n)

In [ ]:
model.fit(train_data, padded_sents)
print(model.vocab)

In [ ]:
len(model.vocab)

In [ ]:
with open('model/task2_sent_p.model','wb') as f:
    pickle.dump(model,f)

### 帮助函数

In [ ]:
def get_new_score(score):
    if score==1:
        return score
    score = '%e' % score
    return (100-int(str(score).split('.')[0])*int(str(score).split('e-')[-1]))/100

def get_p_score(sent, details=False):
    sent = sent.lower()
    words = word_tokenize(sent)
    score = 1
    for i in range(len(words)-1):
        score_item = model.score(words[i+1], [words[i]])
        if details:
            print(f"p({words[i+1]}|{words[i]})={score_item}")
        if score_item == 0:
            continue
        else:
            score = score*score_item
    score = get_new_score(score)
    return score

### 使用

In [ ]:
try: # Use the default NLTK tokenizer.
    from nltk import word_tokenize, sent_tokenize 
    # Testing whether it works. 
    # Sometimes it doesn't work on some machines because of setup issues.
    word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])
except: # Use a naive sentence tokenizer and toktok.
    import re
    from nltk.tokenize import ToktokTokenizer
    # See https://stackoverflow.com/a/25736515/610569
    sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
    # Use the toktok tokenizer that requires no dependencies.
    toktok = ToktokTokenizer()
    word_tokenize = toktok.tokenize

In [ ]:
sent_p_model = pickle.load(open('model/task2_sent_p.model','rb'))

In [ ]:
sent_p_model.score('love',['me'])

In [ ]:
def get_new_score(score):
    if score==1:
        return score
    score = '%e' % score
    return (100-int(str(score).split('.')[0])*int(str(score).split('e-')[-1]))/100

def get_p_score(sent,model=sent_p_model,details=False):
    sent = sent.lower()
    words = word_tokenize(sent)
    score = 1
    for i in range(len(words)-1):
        score_item = model.score(words[i+1], [words[i]])
        if details:
            print(f"p({words[i+1]}|{words[i]})={score_item}")
        if score_item == 0:
            continue
        else:
            score = score*score_item
    score = get_new_score(score)
    return score

In [ ]:
input_js_list = pickle.load(open('data/after_preprocess/task2/input_js_list_gen.pkl','rb'))
len(input_js_list)

In [ ]:
sentence_list = []
for input_js in input_js_list:
    df_hy = pd.DataFrame({"text": input_js['gen_text_list']})
    df_hy['score'] = df_hy['text'].apply(lambda x: get_p_score(x))
    df_hy = df_hy.sort_values("score", ascending=False)
    sentence_list.append(df_hy.iloc[0]['text'])

In [ ]:
df_label = pd.read_csv('data/after_preprocess/task2_df_test_dict.csv')
len(df_label)

In [ ]:
df_label['Sentence'] = sentence_list

In [ ]:
row_list = []
for _,row in df_label.iterrows():
    for speaker in row['Speaker'].split(";"):
        one_line = row.to_dict()
        one_line['Speaker'] = speaker
        row_list.append(one_line)
df_submit = pd.DataFrame(row_list)

In [ ]:
df_submit['Dialogue_id'] = df_submit['Dialogue_id_old']

In [ ]:
df_submit

In [ ]:
keep_cols = ['fname','Dialogue_id','Speaker','Sentence']
week_name = "final2_0923"

In [ ]:
df_submit[keep_cols].to_csv(f'data/TAL-JNU_任务2.csv',index=False)

In [ ]:
df_submit.shape